# BAC Mauritania 2022 Predictive Modeling Challenge

## Importez les librairies usuelles

In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix ,mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay,f1_score
import warnings
warnings.simplefilter("ignore")

### Importez les data et les afficher

In [2]:
test=pd.read_csv("test_set.csv",sep=";")
train=pd.read_csv("train_set.csv")
train.head()

,Unnamed: 0,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,moyeneGeneral,Decision,Note1,...,Note3,Note4,Note5,Note6,Note7,Note8,Willaya,moughataa,DateNaissance,ID
0,27634,El Ide Brahim M'Hadi,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,4.601562,Ajourné,1.625,...,5.00,5.500,3.00,7.75,7.50,18.5,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998,ID_27634
1,36311,Mariem Yedaly Afane,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,7.625000,Ajourné,13.000,...,10.75,2.000,8.75,2.00,0.75,-1.0,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991,ID_36311
2,21059,Aiche Jiyid Jiyid,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,6.894531,Ajourné,7.000,...,6.50,7.625,1.00,5.50,10.00,17.0,Inchiri,AKJOUJT,8 مايو 1995,ID_21059
3,24532,Ahmed Abd El Ghader Meissa,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,4.859375,Ajourné,1.750,...,5.50,5.500,5.00,1.50,7.50,17.0,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999,ID_24532
4,17861,Mariam Abdoulaye Ba,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,3.548387,Ajourné,4.500,...,3.50,4.000,4.00,2.00,2.00,-1.0,Dakhlet NDB,NOUADHIBOU,10 mai 2001,ID_17861


## Afficher la shape du dataset

In [138]:
print(train.shape)
print(test.shape)

(23894, 21)
(15930, 9)


## préparation de données

In [122]:
X=bac.drop(['Decision','DateNaissance','moyeneGeneral', 'Note1',
       'Note2', 'Note3', 'Note4', 'Note5', 'Note6', 'Note7', 'Note8','Unnamed: 0'], axis=1)
Y=pd.DataFrame({'Decision':bac['Decision']})
X.head()

,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,ID
0,El Ide Brahim M'Hadi,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,Nouakchott 2 (Ouest),KSAR,ID_27634
1,Mariem Yedaly Afane,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,Nouakchott 3 (Sud),ARAFAT,ID_36311
2,Aiche Jiyid Jiyid,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,Inchiri,AKJOUJT,ID_21059
3,Ahmed Abd El Ghader Meissa,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,Nouakchott 1 (Nord),TOUJOUNINE,ID_24532
4,Mariam Abdoulaye Ba,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,Dakhlet NDB,NOUADHIBOU,ID_17861


In [123]:
X = X.set_index('ID')
Y=Y.set_index(pd.RangeIndex(start=0, stop=len(Y)))
# Split dataset into training set and test set
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=45)
X_train = X.iloc[0:15930,:] 
X_test = X.iloc[15930:,:]
Y_train =Y.iloc[0:15930,:]
Y_test =Y.iloc[15930:,:]

In [124]:
categorical_cols = ['Nom', 'Serie,x', 'GroupeAnonymat', 'LieuNaissance',
       'Centre', 'Etablissement','Willaya', 'moughataa']
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)])

# fit the preprocessing pipeline on the training data and transform both the training and testing data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
le = LabelEncoder()
Y_train_encoded = le.fit_transform(Y_train)
Y_test_encoded = le.transform(Y_test)
decoded_train=le.inverse_transform(Y_train_encoded)
decoded_test=le.inverse_transform(Y_test_encoded)



# XGBOOST Model

In [140]:
from xgboost import XGBClassifier

# Create an XGBClassifier with CPU acceleration
#learning_rates = [0.05]
xgb = XGBClassifier(tree_method='exact',n_estimators=87,eval_metric='merror', gamma=0.2, reg_alpha=0.2, reg_lambda=0.1)
# Train the model and make predictions
xgb.fit(X_train_preprocessed, Y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, gamma=0.2, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=87, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [141]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Create an XGBClassifier with CPU acceleration and early stopping
xgb = XGBClassifier(tree_method='exact', n_estimators=105, eval_metric='merror', gamma=0.2, reg_alpha=0.2, reg_lambda=0.1, early_stopping_rounds=500)

# Train the model and make predictions
xgb.fit(X_train_preprocessed, Y_train_encoded, eval_set=[(X_test_preprocessed, Y_test_encoded)], verbose=False)

# predict with the model 
y_pred = xgb.predict(X_test_preprocessed)

# Find the SUCCESS probability
y_pred_proba = xgb.predict_proba(X_test_preprocessed)
probability=[y_pred_proba[i][1] for i in range(len(y_pred_proba))]
accuracy = accuracy_score(Y_test_encoded, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7985936715218483


In [77]:
#test = test.set_index('ID')
test= test.drop(columns=['Unnamed: 0','prediction'])
import pandas as pd

sub=pd.read_csv("C:/Users/hp/Desktop/DataChallenge/submission_template.csv")




In [126]:
test= test.drop(columns=['Unnamed: 0','prediction'])
import pandas as pd
sub=pd.read_csv("C:/Users/hp/Desktop/DataChallenge/submission_template.csv")

In [127]:
in_test= sub['Id'].isin(test['ID'])
in_train= sub['Id'].isin(train['ID'])
test_fil= test[test['ID'].isin(sub['Id'])]
train_fil = train[train['ID'].isin(sub['Id'])]
mrg= pd.concat([test_fil, train_fil])
mrg = mrg.drop(columns=[
'DateNaissance','moyeneGeneral', 'Decision', 'Note1', 'Note2', 'Note3', 'Note4','Note5', 'Note6', 'Note7', 'Note8'
])
test= test.reindex(columns = ['ID', 'Nom', 'Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre',
       'Etablissement', 'Willaya', 'moughataa'])

In [128]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)])

# fit the preprocessing pipeline on the training data
preprocessor.fit(X_train)

# transform both the training and testing data using the fitted transformer
X_train_preprocessed = preprocessor.transform(X_train)
test_preprocessed = preprocessor.transform(matches)

print(X_train_preprocessed.shape)
print(test_preprocessed.shape)



(15930, 16853)
(15930, 16853)


## Proba de Prediction

In [129]:
y_pred_proba =xgb.predict_proba(test_preprocessed)
proba= [y_pred_proba[i][1] for i in range(len(y_pred_proba))]

In [136]:
sub= pd.DataFrame(mrg['ID'])
sub['Predicted'] = proba
sub['Predicted']

1        0.049560
6        0.049560
7        0.049560
11       0.049560
13       0.049560
           ...   
23884    0.017436
23885    0.005420
23886    0.010015
23888    0.024797
23890    0.030581
Name: Predicted, Length: 15930, dtype: float32

### Sauvegarder le DataFrame sous la forme d'un fichier CSV

In [139]:
sub.to_csv('submission_template.csv',index=False)